In [10]:
#import necessary librarires 
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import nltk
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
import re 

In [20]:
#Download necessary NLTK data 
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\muthu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\muthu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\muthu\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [21]:
# Load the train and test datasets 
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [22]:
train.head()

,medicine_no,disease_type,medicine_review,market_value,launch_date,score
0,MD-22117,677,I read so many horrible reviews of this that I...,10,19-Mar-16,19
1,MD-22118,121,"When I took the pill, I had a hard time rememb...",8,28-Oct-13,21
2,MD-22119,200,This is the only antidepressant that was bene...,9,01-Jun-17,34
3,MD-22120,455,I took Alli for 2 months when it first came ou...,7,13-May-08,98
4,MD-22121,404,About ten days or two weeks after starting Est...,10,13-Dec-09,43


In [23]:
test.head()

,medicine_no,disease_type,medicine_review,market_value,launch_date
0,MD-49997,555,I came back from Cuba with a sinus infection a...,10,23-Jan-14
1,MD-49998,472,I just had my first injection both knees a few...,10,17-Jun-16
2,MD-49999,308,Doesn&#039;t work to lower blood pressure. I s...,2,25-Jul-15
3,MD-50000,352,"TRAZODONE, gives nasty neck &amp; headaches to...",1,16-Jun-16
4,MD-50001,121,I was on this pill for 4 years. The side effec...,3,01-Aug-15


In [24]:
# Setup stopwords and lemmatizer for text processing 
stopwords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

AttributeError: 'set' object has no attribute 'words'

In [25]:
# Function to preprocess text: lowercasing, removing punctuation, stopwords, and lemmatizing
def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())  # Remove punctuation and convert to lowercase
    text = [lemmatizer.lemmatize(word) for word in text.split() if word not in stopwords]  # Lemmatize and remove stopwords
    return ' '.join(text)  # Join the words back into a single string

# Apply text preprocessing to the medicine_review column in both train and test datasets
train['processed_review'] = train['medicine_review'].apply(preprocess_text)
test['processed_review'] = test['medicine_review'].apply(preprocess_text)

In [27]:
 # Convert the processed text reviews into numerical features using TF-IDF vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train = tfidf_vectorizer.fit_transform(train['processed_review']).toarray()  # Fit and transform on training data
X_test = tfidf_vectorizer.transform(test['processed_review']).toarray()  # Transform test data based on training data
y_train = train['score']  # Target variable


In [28]:
 # Train a Linear Regression model on the training data
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [29]:
# Evaluate the model using RMSE on the training set (for demonstration purposes)
y_pred_train = model.predict(X_train)
rmse_train = mean_squared_error(y_train, y_pred_train, squared=False)
print(f"RMSE on training set: {rmse_train}")

RMSE on training set: 28.678384811643166


In [30]:
# Predict usefulness scores for the test data
predictions = model.predict(X_test)

In [31]:
# Prepare the submission dataframe with medicine_no and predicted scores
submission = pd.DataFrame({
    'medicine_no': test['medicine_no'],
    'predicted_score': predictions
    })


In [33]:
# Save the submission dataframe to a CSV file
submission.to_csv('submission.csv', index=False)